In [1]:
import random

In [39]:
class Die:
    def __init__(self):
        self.value = None

    def __repr__(self):
        return str(self.value)

    def roll(self):
        self.value = random.randint(1,6)

In [52]:
class Player:
    def __init__(self, name):
        self.name = name
        self.dice = [Die() for i in range(6)]

    def __repr__(self):
        repr = f"Player: {str(self.name)}\n"
        repr += f"Dice: {str(self.show_dice_values())}"


    def get_name(self):
        return self.name
    
    def get_dice(self):
        return self.dice

    def roll_dice(self):
        for die in self.dice:
            die.roll()

    def show_dice_values(self):
        l = []
        for die in self.dice:
            l.append(die.get_value())
        return l

In [93]:
class Bid:
    def __init__(self, quantity, value):
        self.quantity = quantity
        self.value = value

    def __eq__(self, other):
        return self.quantity == other.quantity and self.value == other.value

    def __gt__(self, other):
        if self.notAceBid() and other.notAceBid() :
            if self.quantity == other.quantity :
                return self.value > other.value
            else :
                return self.quantity > other.quantity
        if self.isAceBid() and other.notAceBid() : 
            return True # Need to implement
        if self.notAceBid() and other.isAceBid() : 
            return False # Need to implement
        if self.isAceBid() and other.isAceBid() :
            return self.quantity > other.quantity

    def __lt__(self, other):
        if self.notAceBid() and other.notAceBid() : 
            if self.quantity == other.quantity :
                return self.value < other.value
            else :
                return self.quantity < other.quantity
        if self.isAceBid() and other.notAceBid() : 
            # Implement this
            return False
        if self.notAceBid() and other.isAceBid() :
            # Implement this too
            return True
        if self.isAceBid() and other.isAceBid() :
            return self.quantity < other.quantity

    def isAceBid(self):
        return self.value == 1

    def notAceBid(self):
        return not(self.isAceBid())


In [95]:
b1 = Bid(3, 3)
b2 = Bid(4, 5)

b1.isAceBid()

False